<h1>Tuto prise en main de pinocchio</h1>

Dans ce tutorial nous allons essayer de voir quelques petites manipulations de base que l'on peut faire avec Pinocchio.
Tout d'abords Pinocchio est une bibliothèque permettant de calculer efficacement la dynamique (et les dérivés) d’un modèle de robot, ou de tout modèle articulé à corps rigide que vous souhaitez.
<h2><li>Charger le modèle du robot</li></h2>
La première des choses que vous avez à faire est de charger le modèle du robot. Pour cela vous pouvez déjà regarder si le modèle avec lequel vous voulez travailler ne se trouve pas déjà dans la base d’exemple (« example_robot_data ») qui vient avec l’installation de Pinocchio. Sinon vous devez juste trouver le chemin absolu de votre fichier «.urdf » qui se trouve dans le dossier de description de votre robot. 
Déjà nous allons importer les packages qui nous seront nécessaire 


In [82]:
import pinocchio as pin
from example_robot_data import load
from pinocchio.robot_wrapper import RobotWrapper
from numpy.linalg import norm
import numpy

Si le modèle que vous voulez charger se trouve dans la base d'exemple déjà présente vous pouvez juste utilisez la méthode load importé qui prend paramètre le nom du robot que vous voulez charger. Par exemple on va charger le bras robotique ur5 qui se trouve déjà dans la base d'exemple juste en faisant :  

In [83]:
robot = load('ur5')

Pour trouver ces noms de robots qui sont pris en paramètre vous pouvez juste mettre n’import quel caractère en paramètre de la fonction load et un message d’erreur sera
produit avec la liste des noms de robots que vous pouvez charger.

In [84]:
robot = load('e')

ValueError: Robot 'e' not found. Possible values are a1, anymal, anymal_kinova, asr_twodof, baxter, bolt, cassie, double_pendulum, double_pendulum_continuous, double_pendulum_simple, finger_edu, hector, hyq, icub, icub_reduced, iris, kinova, laikago, panda, romeo, simple_humanoid, simple_humanoid_classical, solo12, solo8, talos, talos_arm, talos_box, talos_full, talos_full_box, talos_legs, tiago, tiago_dual, tiago_no_hand, ur10, ur10_limited, ur3, ur3_gripper, ur3_limited, ur5, ur5_gripper, ur5_limited

Et si vous voulez charger un modèle qui ne se trouve pas dans cette liste vous pouvez utilisez la méthode 
RobotWrapper.BuildFromURDF qui prend en paramètre le chemin du fichier urdf énoncé plus haut ainsi que le chemin du répertoire du package
Juste pour illustrer nous allons charger l'ur5 mais avec le fichier urdf

In [85]:
URDF = '/opt/openrobots/share/example-robot-data/robots/ur_description/urdf/ur5_robot.urdf'
robot = RobotWrapper.BuildFromURDF(URDF,['/opt/openrobots/share'])

Après le chargement du modèle, un objet de type RobotWrapper est stocké dans la variable "robot". Et ce type contient beaucoup d'attributs et de mèthodes dont certains nous servirons pour la suite.

<h3><li>Visualiser le robot</li></h3>
Maintenant que nous avons chargé le robot nous avons la possibilité de visualiser le modèle dans gepetto-gui.
Pour cela ouvrez le en tapant gepetto-gui dans le terminale et vous verrez une fenêtre apparaîte.
Ensuite faite :

In [86]:
robot.initViewer(loadModel = True) 

Pour bien pouvoir voir le bras nous pouvons l'afficher avec une configuration précise en faisant :

In [87]:
robot.display(robot.q0) # robot.q0 est un vecteur de l'espace des configuration de l'instance robot

A noter que la méthode display prend n'importe quelle vecteur de configuration ayant les bonnes dimensions en paramètre. 

Vous avez aussi la possibilité d'ajouter des objets géométriques dans le visualiseur. Nous allons essayer d'ajouter une sphére par exemple: 

In [88]:
rgbt = [0.2, 1.2, 0.2, 1.0]  # rouge, vert, bleu, transparence (la couleur de l'objet)
robot.viewer.gui.addSphere("world/target", .05, rgbt)  # .05 represente le rayon de la sphère t
target= [0.7,-0.3,0.4]       # Coordonnée de la sphère
q1 = (target[0], target[1],target[2], 1, 0, 0, 0)  # x, y, z, quaternion
robot.viewer.gui.applyConfiguration("world/target", q1) # Création de la sphère
robot.viewer.gui.refresh()  # Pour rafraîchir la fenêtre.

<h2><li>Algorithmes implémentés dans pinocchio</li></h2>
Pour une configuration donnée il nous est possible de calculer l'emplacement des joints en utilisant :pinocchio.forwardKinematics elle prend en paramètre le modèle du robot qui se trouve dans robot.model, le champs robot.data là où sont stockés les résultats de certains algorithmes implémentés dans pinocchio et la configuration.

Pour l'exemple que nous allons voir, nous allons prendre une configuration aléatoire , la dimension du vecteur de configuration se trouve dans robot.model.nq. Le nombre du degrée de liberté est robot.model.nv.


In [89]:
q = numpy.random.rand(robot.model.nq)

Pour avoir une configuration aléatoire vous pouvez aussi faire :

In [90]:
q = pin.randomConfiguration(robot.model)

In [91]:
pin.forwardKinematics(robot.model,robot.data,q)

Les emplacements des joints ainsi calculés sont dant robot.data.oMi.

In [92]:
print(robot.data.oMi[0])  # emplacement du joint d'indice 0 (la base)

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0



Faisons un petit focus sur robot.model : 

In [93]:
print(robot.model)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



Il contient des inforamtions sur les joints (leur parent, leur indice , leur nom)

Il est aussi possible d'obtenir l'emplacement de chaque repère mais pour cela même après le calcul de la cinématique directe il faut mettre à jour leur emplacement en faisant

In [94]:
pin.updateFramePlacements(robot.model,robot.data)

Les emplacements des repères sont stockés dans robot.data.oMf[]

In [95]:
print(robot.data.oMf[0]) #Emplacement du repère d'indice 0 (repère de base) 

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0



Vous pouvez aussi faire :

In [96]:
print(robot.model.frames[0].placement)

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0



In [97]:
print(robot.model.frames[0].placement.translation) # Attribut translation pour avoir juste la partie translation de l'élément de SE3 
print(robot.model.frames[0].placement.rotation)  # Attribut rotation pour avoir juste la partie rotation de l'élément de SE3

[0. 0. 0.]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Pour avoir le nom des frames(repères) et ceux de leurs parents :

In [98]:
for i,f in enumerate(robot.model.frames): print(i,f.name,f.parent)

0 universe 0
1 root_joint 0
2 world 0
3 world_joint 0
4 base_link 0
5 base_link-base_fixed_joint 0
6 base 0
7 shoulder_pan_joint 1
8 shoulder_link 1
9 shoulder_lift_joint 2
10 upper_arm_link 2
11 elbow_joint 3
12 forearm_link 3
13 wrist_1_joint 4
14 wrist_1_link 4
15 wrist_2_joint 5
16 wrist_2_link 5
17 wrist_3_joint 6
18 wrist_3_link 6
19 ee_fixed_joint 6
20 ee_link 6
21 wrist_3_link-tool0_fixed_joint 6
22 tool0 6


Pour trouver l'indice du repère dont vous avez le nom utilisez getFrameId de robot.model : 

In [99]:
Id = robot.model.getFrameId("shoulder_link")
print(Id)

8


Par la suite nous allons voir certains calculs différentiels avec pinocchio.
Si nous avons par exemple une configuration de départ ainsi qu'une vitesse de départ il nous est possible de trouver la prochaine configuration au bout d'une seconde en faisant : 

In [100]:
qdepart = pin.randomConfiguration(robot.model)# vecteur configuration
v0 = numpy.random.rand(robot.model.nv) # vecteur vitesse
qnext = pin.integrate(robot.model,qdepart,v0)
print(qnext)

[ 4.68558946  5.73161098 -2.1233011   6.09348925  0.62111011 -4.83784577]


Si par contre vous avez une configuration de départ et une d'arrivée et vous voulez connaître la vitesse au bout d'une seconde :

In [101]:
qdepart = pin.randomConfiguration(robot.model)
qarrivee = pin.randomConfiguration(robot.model)
vq = pin.difference(robot.model,qdepart,qarrivee)
print(vq)

[ 3.33621924 -7.54146968 -4.0963393   7.8130303  10.53108042 10.44135635]


Par ailleur, dans le modèle cinématique vous aurez à calculer la jacobienne d'un joint ou d'un repère. Pour cela , calculez d'abords l'emplacement des joints, puis la jocobienne de chaque joint  avec pinocchio.computeJointjacobian.  

In [102]:
pin.forwardKinematics(robot.model,robot.data,qdepart)
pin.computeJointJacobians(robot.model,robot.data,qdepart)

array([[ 0.        ,  0.0666667 , -0.21515166,  0.07174307, -0.19601762,
         0.27493487],
       [ 0.        , -0.05920201,  0.19106109, -0.06370999, -0.07952145,
         0.06410726],
       [ 0.        ,  0.        , -0.19639702, -0.27790267,  0.05587305,
        -0.15818959],
       [ 0.        , -0.66400492, -0.66400492, -0.66400492,  0.38275636,
         0.00407985],
       [ 0.        , -0.7477282 , -0.7477282 , -0.7477282 , -0.33989906,
        -0.92922803],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.85904959,
        -0.36948428]])

Pour avoir la jacobienne des repères pensez d'abords à mettre leur emplacement à jour puis utilisez pinocchio.getFrameJacobian 
prend en paramètre le modèle, les données, l’index du repère que
l’on a évoqué plus haut, puis le repère de référence par rapport auquel vous voulez calculer votre jacobienne.

Pour le repère de référence on a :

In [103]:
LOCAL = pin.ReferenceFrame.LOCAL
print(LOCAL)

LOCAL


Ou encore :

In [104]:
WORLD = pin.ReferenceFrame.WORLD
print(WORLD)

WORLD


In [105]:
pin.updateFramePlacements(robot.model,robot.data)
pin.getFrameJacobian(robot.model,robot.data,22,LOCAL) # 22 indice du repère de l'organe terminal(univers)

array([[-3.80639572e-01,  8.93474632e-02,  3.29027042e-01,
         4.86547126e-02, -7.97084627e-02,  0.00000000e+00],
       [ 1.00541932e-02, -3.27209450e-01, -1.54178519e-02,
        -7.38430185e-02,  2.04902654e-02,  0.00000000e+00],
       [-1.58189591e-01, -1.75208752e-01, -3.36344404e-01,
        -6.83184369e-02, -1.00332936e-13,  0.00000000e+00],
       [ 1.29247340e-01,  6.99071990e-01,  6.99071990e-01,
         6.99071990e-01, -2.48970418e-01,  2.77555756e-17],
       [-9.20204593e-01, -1.79707024e-01, -1.79707024e-01,
        -1.79707024e-01, -9.68511090e-01,  4.89669416e-12],
       [-3.69484278e-01,  6.92100960e-01,  6.92100960e-01,
         6.92100960e-01,  4.74237316e-12,  1.00000000e+00]])

En plus de ça certains algorithmes en lien avec la dynamique sont implémentés dans pinocchio par exemple RNEA pour la dynamique inverse, ABA pour la dynamique directe, le CRBA pour la matrice de masse

In [106]:
q = pin.randomConfiguration(robot.model) # Configuration aléatoire
vq = numpy.random.rand(robot.model.nv)*2-1 # Vecteur vitesse
aq = numpy.random.rand(robot.model.nv)*2-1 # vecteur accélération

In [107]:
tauq = pin.rnea(robot.model,robot.data,q,vq,aq) # dynamique inverse
print(tauq)

[ 2.56777353e+00 -5.01242674e+01 -1.40698803e+01  3.21933407e-01
 -1.54459182e-02  1.62398433e-02]


Vérifions si le calcul de la dynamique directe avec la dynamique inverse que nous venons de trouver correspond bien à l'accélération aq utilisé avant

In [108]:

aq1 = pin.aba(robot.model,robot.data,q,vq,tauq)
print(norm(aq-aq1))

1.8590849472252627e-14


In [109]:
M = pin.crba(robot.model,robot.data,q) #Matrice de masse du robot
print(M)

[[ 3.68945116e+00  1.71309909e-01 -2.44071137e-02 -2.75827739e-03
   2.36885332e-01 -4.16779713e-04]
 [ 1.71309909e-01  3.56199529e+00  1.31539037e+00  2.37758290e-01
  -1.15040051e-03  1.69243017e-02]
 [-2.44071137e-02  1.31539037e+00  8.38912388e-01  2.42364475e-01
  -1.15040051e-03  1.69243017e-02]
 [-2.75827739e-03  2.37758290e-01  2.42364475e-01  2.41468514e-01
  -1.15040051e-03  1.69243017e-02]
 [ 2.36885332e-01 -1.15040051e-03 -1.15040051e-03 -1.15040051e-03
   2.40921257e-01  0.00000000e+00]
 [-4.16779713e-04  1.69243017e-02  1.69243017e-02  1.69243017e-02
   0.00000000e+00  1.71364731e-02]]


La fonction pinocchio.computeAllTerm() permet de calculer le terme  b(q, vq) = c(q, vq) + g(q) et le stocke dans
robot.data.nle .Elle calcul aussi en même temps la matrice de masse M et le stocke dans robot.data.M

In [110]:
pin.computeAllTerms(robot.model,robot.data,q,vq)

Le terme b(q,vq) peut être aussi calculer en annulant l'accélération dans l'algorithme du rnea

In [111]:
b = pin.rnea(robot.model,robot.data,q,vq,aq*0)

On peut bien vérifier pour voir si cela correspond bien à ce qu'il y'a dans robot.data.nle : 

In [112]:
print(norm(b-robot.data.nle))

0.0


Faisons de même pour M et robot.data.M : 

In [113]:
print(norm(M - robot.data.M))

6.555614292786558e-16


<h2><li>Collisions</li></h2> 
A prèsent nous allons parler des collisions dans pinocchio.
En effet nous avons la possibilité de placer des obstacles dans l'environement du robot.
Dans la classe GeometryObject
de pinocchio vous pouvez par exemple crée un obstacle en forme de capsule avec la fonction CreateCapsule qui prend en paramètre le rayon et la
longueur de la capsule. Cela vous renvoie un objet dont vous avez à paramètrer la couleur (obstacle.meshColor = [1.0 ,0.2, 0.3 ,1.0]) qui est un vecteur de la forme rgb transparence, le nom (obstacle.name = ’obstacle’), l’objet parent (obstacle.parentJoint = 0 avec 0 indice de l’univers), l’emplacement de
l’objet (obstacle.placement = matrice SE3).

Pour crée notre obstacle nous allons chargé ur5 dans une autre variable robot2. C'est mieux de créer les obstacles d'abords avant d'ajouter autre chose dans l'environnement du robot

In [114]:
robot2 = load('ur5')

In [115]:
color = [ 1.0, 0.2, 0.2, 1.0 ]                       # couleur de la capsule
rad,length = .1,0.4                                  # rayon et longueur de la capsule
R = numpy.array([[1.,0.,0.],[0.,0.,-1.],[0.,1.,0.]])  # Matrice de rotation
P = numpy.array([0.6, 0., 0.2 ])                    # Vecteur de translation
obs = pin.GeometryObject.CreateCapsule(rad,length)  # Pinocchio obstacle object
obs.meshColor = numpy.array(color)    # Couleur de l'obstacle 
obs.name = "obstacle"                             # nom de l'objet
obs.parentJoint = 0                                 # objet parent = 0 = universe
obs.placement = pin.SE3(R,P)  # fonction permettant de d'avoir un élément de SE3


Après la création de l'objet géométrique vous devez l'ajouter dans le modèle visuel et de collision du robot :

In [116]:
robot2.collision_model.addGeometryObject(obs)  # Ajout de l'objet dans le modèle de  collision
robot2.visual_model.addGeometryObject(obs)  # Ajout de l'objet dans le modèle de visuel

7

Maintenant il nous reste à créer les pairs de collision grâce à la méthode pinocchio.collisionPair qui prend en paramêtre les indices des objets géométriques.
Chaque partie du robot doit former une paire de collision avec chaque obstacles.

In [117]:
robotbodies = range(robot2.collision_model.ngeoms - 1) # le nombre de corps du robot correspond au nombre d'objet géométrique dans le modèle de collision moins le nombre d'obstacle 
                                                      # Mais ici nous avons besoin des indices de ces objets géométriques pour créer des paires de collision
obsIndice = robot2.collision_model.ngeoms - 1          # Indice de notre obstacle (Dans notre cas c'est le dernier objet géométrique)

Par précaution on enlève toutes les paires de collision qui existait et nous ajouterons celles que nous auront créées

In [118]:
robot2.collision_model.removeAllCollisionPairs()
for i in robotbodies :
    robot2.collision_model.addCollisionPair(pin.CollisionPair(i,obsIndice))

Par la suite il nous faut générer les données de collision correspondant

In [119]:
robot2.collision_data = pin.GeometryData(robot2.collision_model)
robot2.visual_data = pin.GeometryData(robot2.visual_model)

Pour pouvoir visualiser le bras robotique avec l'obstacle fermer la fenêtre de gepetto-gui et relancer la commande sur le terminal.

In [120]:
robot2.initViewer(loadModel = True)
robot2.display(robot2.q0)

Nous pouvons ajouter notre sphère comme nous avons fait plutôt 

In [121]:
rgbt = [0.2, 1.2, 0.2, 1.0]  # rouge, vert, bleu, transparence (la couleur de l'objet)
robot2.viewer.gui.addSphere("world/target", .05, rgbt)  # .05 représente le rayon de la sphère 
target= [0.7,-0.3,0.4]       # Coordonnée de la sphère
q1 = (target[0], target[1],target[2], 1, 0, 0, 0)  # x, y, z, quaternion
robot2.viewer.gui.applyConfiguration("world/target", q1) # Création de la sphère
robot2.viewer.gui.refresh()  # Pour rafraîchir la fenêtre.

Avec Pinocchio, nous avons la possibilité de vérifier si le robot est en collision avec les obstacles ajoutés dans son environnement. Pour cela nous devons d’abord mettre à jour l’emplacement de l’ensemble des objets géométriques de l’environnement en faisant :

In [122]:
pin.updateGeometryPlacements(robot2.model,robot2.data,robot2.collision_model,robot2.collision_data,robot2.q0) # on met en dernier paramètre la configuration que l'on souhaite tester la collision

Puis pour tester s'il y'a collision ou pas on utilise la fonction pin.computeCollisions() : 

In [123]:
pin.computeCollisions(robot2.collision_model, robot2.collision_data,False) 

True

Si la valeur du dernier paramètre  est True, alors la fonction arrête la boucle sur les paires de collisions lorsque la première collision est détectée et alors le vecteur collisions ne sera pas entièrement rempli.

La fonction renvoie True s'il y'a collision et False sinon